In [2]:
from myosuite.utils import gym as myo_gym
from config import Config
import numpy as np

MyoSuite:> Registering Myo Envs


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {ne

In [3]:
cfg = Config()

# 1. Create environment
env = myo_gym.make(cfg.env_id)

# 2. Reset environment (returns array, not dict)
obs, info = env.reset()

print("RAW obs returned by reset():")
print(type(obs), obs.shape)

/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    
RAW obs returned by reset():
<class 'numpy.ndarray'> (417,)


In [4]:
# 3. Get the TRUE dict-based observation
if hasattr(env, "obs_dict"):
    obs_dict = env.obs_dict
    print("\nOBS_DICT KEYS:")
    for k, v in obs_dict.items():
        print(f"{k:20s} shape={np.array(v).shape}")
else:
    print("ERROR: env.obs_dict not found")



OBS_DICT KEYS:
time                 shape=(1,)
pelvis_pos           shape=(3,)
body_qpos            shape=(58,)
body_qvel            shape=(58,)
ball_pos             shape=(3,)
ball_vel             shape=(3,)
paddle_pos           shape=(3,)
paddle_vel           shape=(3,)
paddle_ori           shape=(4,)
padde_ori_err        shape=(4,)
reach_err            shape=(3,)
palm_pos             shape=(3,)
palm_err             shape=(3,)
touching_info        shape=(6,)
act                  shape=(273,)


/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obs_dict to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.obs_dict` for environment variables or `env.get_wrapper_attr('obs_dict')` that will search the reminding wrappers.
  logger.warn(
